In [1]:
import mysql.connector
import ttkbootstrap as ttk
from ttkbootstrap.constants import *
from tkinter import messagebox
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker for generating random data
fake = Faker()

# MySQL Database Connection Details
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "vedant",
    "database": "crop_management_system"
}

# Sample data
crop_names = ["Wheat", "Rice", "Corn", "Soybean", "Barley", "Sugarcane", "Cotton", "Potato", "Tomato", "Lettuce"]
growth_stages = ["Seedling", "Vegetative", "Flowering", "Fruiting", "Maturity"]
pest_control_measures_list = [
    "Use of organic pesticides", "Crop rotation", "Neem oil application", 
    "Biological pest control", "Chemical pesticides", "Regular field monitoring"
]

# Database Connection Function
def connect_db():
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        return conn
    except mysql.connector.Error as e:
        messagebox.showerror("Database Error", f"Error connecting to database: {e}")
        return None

# Function to Insert Manual Crop Record
def insert_manual_record():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        values = (crop_name_var.get(), planting_date_var.get(), harvest_date_var.get(), 
                  growth_stage_var.get(), pest_control_var.get(), yield_var.get())
        
        if any(not v for v in values):
            messagebox.showwarning("Input Error", "All fields must be filled!")
            return
        
        try:
            cursor.execute("""
                INSERT INTO crops (crop_name, planting_date, harvest_date, growth_stage, pest_control_measures, yield_prediction)
                VALUES (%s, %s, %s, %s, %s, %s)
            """, values)
            conn.commit()
            messagebox.showinfo("Success", "Crop record inserted successfully!")
            conn.close()
            display_records()
        except mysql.connector.Error as e:
            messagebox.showerror("Database Error", f"Error inserting record: {e}")

# Function to Generate Random Data for Bulk Insert
def generate_data():
    planting_date = fake.date_between(start_date="-2y", end_date="today")
    harvest_date = planting_date + timedelta(days=random.randint(60, 180))
    return (random.choice(crop_names), planting_date, harvest_date, random.choice(growth_stages), 
            random.choice(pest_control_measures_list), random.randint(500, 5000))

# Function to Insert Bulk Records
def insert_bulk_records():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        batch_size = 10000
        total_records = 100000
        
        for i in range(0, total_records, batch_size):
            cursor.executemany("""
                INSERT INTO crops (crop_name, planting_date, harvest_date, growth_stage, pest_control_measures, yield_prediction)
                VALUES (%s, %s, %s, %s, %s, %s)
            """, [generate_data() for _ in range(batch_size)])
            conn.commit()
            progress_label.config(text=f"{i + batch_size} records inserted...")
        
        messagebox.showinfo("Success", "100,000 records inserted successfully!")
        conn.close()
        display_records()

# Function to Display Records
def display_records():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM crops ORDER BY id DESC LIMIT 20")
        rows = cursor.fetchall()
        conn.close()
        tree.delete(*tree.get_children())
        for row in rows:
            tree.insert("", "end", values=row)

# GUI Setup
root = ttk.Window(themename="superhero")
root.title("Crop Management System")
root.geometry("900x800")

# Title
title_label = ttk.Label(root, text="Crop Management System", font=("Arial", 20, "bold"), bootstyle=PRIMARY)
title_label.pack(pady=10)

# Input Fields
frame = ttk.Frame(root, padding=10)
frame.pack(pady=10)

fields = ["Crop Name", "Planting Date (YYYY-MM-DD)", "Harvest Date (YYYY-MM-DD)", "Growth Stage", "Pest Control Measures", "Yield Prediction (kg)"]
variables = [ttk.StringVar() for _ in fields]
crop_name_var, planting_date_var, harvest_date_var, growth_stage_var, pest_control_var, yield_var = variables

for i, field in enumerate(fields):
    ttk.Label(frame, text=field, font=("Arial", 12, "bold"), bootstyle=INFO).grid(row=i, column=0, sticky="w", padx=5, pady=5)
    ttk.Entry(frame, textvariable=variables[i], width=30).grid(row=i, column=1, padx=5, pady=5)

# Buttons
button_frame = ttk.Frame(root, padding=10)
button_frame.pack()

ttk.Button(button_frame, text="Insert Record", bootstyle=SUCCESS, command=insert_manual_record).grid(row=0, column=0, padx=10)
ttk.Button(button_frame, text="Insert 100,000 Records", bootstyle=DANGER, command=insert_bulk_records).grid(row=0, column=1, padx=10)

progress_label = ttk.Label(root, text="", font=("Arial", 10))
progress_label.pack(pady=5)

# Table to Display Records
columns = ("ID", "Crop Name", "Planting Date", "Harvest Date", "Growth Stage", "Pest Control", "Yield Prediction")
tree = ttk.Treeview(root, columns=columns, show="headings", height=10, bootstyle=INFO)
for col in columns:
    tree.heading(col, text=col)
    tree.column(col, width=100)
tree.pack(pady=10)

# Load initial records
display_records()

# Run the GUI
root.mainloop()
